In [38]:
import os
import gzip
import xml.etree.ElementTree as ET
import pandas as pd
import random
import numpy as np
import csv
import tempfile

In [39]:
# Set random number
random.seed(42)

# Define the directory where the gz files are located
gz_directory = './pubmed_data'

# List all gz files in the directory
gz_files = [os.path.join(gz_directory, file) for file in os.listdir(gz_directory) if file.endswith('.gz')]

# Randomly select 5 gz files
selected_gz_files = random.sample(gz_files, 5)

In [41]:
# Writing data to CSV file
with open(csv_file, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['pmid', 'title', 'date', 'abstract', 'keywords', 'mesh_descriptor', 'mesh_qualifier', 'xml_file']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    # Iterate through randomly selected files
    for gz_file in selected_gz_files:
        # Unzip files into temporary files
        with gzip.open(gz_file, 'r') as gz:
            temp_file = tempfile.TemporaryFile()
            temp_file.write(gz.read())
            temp_file.seek(0)

            # Store xml structure
            tree = ET.parse(temp_file)
            root = tree.getroot()

            temp_file.close()

            # Interate through xml file and store information
            for article in root.findall('.//PubmedArticle'):
                language = article.find('.//Language').text if article.find('.//Language') is not None else None
                abstract = article.find('.//AbstractText').text if article.find('.//AbstractText') is not None else None

                # Filter for english texts with abstracts
                if language and language.lower() == 'eng' and abstract:
                    
                    # Store pmid as integer
                    pmid = int(article.find('.//PMID').text)
                    
                    # Store all elements of article title
                    article_title_element = article.find('.//ArticleTitle')

                    # Clean HTML tags
                    if article_title_element is not None:
                        article_title = ET.tostring(article_title_element, method="text", encoding="unicode").strip()
                    else:
                        article_title = None

                    title = article.find('.//ArticleTitle').text if article.find('.//ArticleTitle') is not None else None

                    # Clean HTML tags
                    abstract_element = article.find('.//AbstractText')
                    if abstract_element is not None:
                        abstract_text = ET.tostring(abstract_element, method="text", encoding="unicode")
                    else:
                        abstract_text = None

                    # Check for different publication dates
                    ## PubDate: publication date
                    ## DateCompleted: Date revision process completed
                    ## DateRevised: Date of last revision if not yet published
                    date_options = [
                        article.find('.//PubMedPubDate/Year'),
                        article.find('.//DateCompleted/Year'),
                        article.find('.//DateRevised/Year')
                    ]
                    
                    date_created = None
                    for date_option in date_options:
                        if date_option is not None:
                            date_created = int(date_option.text)
                            break
                    
                    keywords_list = [keyword_list.text for keyword_list in article.findall('.//KeywordList/Keyword') if keyword_list.text]
                    keywords = ', '.join(keywords_list) if keywords_list else None
                    
                    mesh_descriptor_list = [mesh_heading.text for mesh_heading in article.findall('.//MeshHeading/DescriptorName')]
                    mesh_descriptor = ', '.join(mesh_descriptor_list) if mesh_descriptor_list else None
                    
                    mesh_qualifier_list = [mesh_heading.text for mesh_heading in article.findall('.//MeshHeading/QualifierName')]
                    mesh_qualifier = ', '.join(mesh_qualifier_list) if mesh_qualifier_list else None
                    
                    # Store name of xml file for manual inspection
                    xml_file = gz_file.split(os.path.sep)[-1]

                    # Write to csv
                    writer.writerow({
                        'pmid': pmid,
                        'title': article_title,
                        'date': date_created,
                        'abstract': abstract_text,
                        'keywords': keywords,
                        'mesh_descriptor': mesh_descriptor,
                        'mesh_qualifier': mesh_qualifier,
                        'xml_file': xml_file
                    })

FileNotFoundError: [WinError 2] Das System kann die angegebene Datei nicht finden: 'C:\\Users\\tassi\\AppData\\Local\\Temp\\tmpk_gp2q62'